# 一、语言与语法

1. 首先需要构建一个简单的便于形成迭代模型的语言定义
1. 而后需要构建语言转换规则，使得可以应用于成为优化调节
1. 不进行解题，而是只用于优化，即问题的化解

# 二、以排序为例

## 1. 输入
Input: x[5]

## 2. 处理
### 做或是求解 ，先考虑Do，优化的Do，而不考虑反解问题
Do or Slove: 

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)
``` 

## 3. 输出
Output: x[5]

# 3.1 模型重构

## 1 语言实现v0.1

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)
``` 

####  问题列表
1. 所有的数据是一个集合，如x_1,x_2,x
1. 操作是对集合的遍历导致的操作，或者对集合的元素进行操作
1. 函数是需要有输入，有输出
1. 函数出入参的传递是生存周期问题
1. 操作是有目的性的，目的性决定了优化性
1. 变量的产生需要定义，需要销毁
1. 表述，及意义在 代码的表达形式是次要的，内涵是一致的


## 2 语言实现v0.2


``` python
for x_1 in range(x.len):
 for x_2 in range(x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  if(x_1<x_2 and x[x_2]<x[x_1]):
    x_t=x[x_i]
    x[x_i]=x[x_k]
    x[x_k]=x_t
```


#### 问题解决列表
1. 层次结构转为==>函数调用结构
1. 消除了变量的生存期，生存期只在函数内
1. 理论上也支持递归
1. 只有层级化解为调用关系，函数内存在执行顺序(依赖顺序，而不是严格顺序),依赖顺序是由于变量变化导致的顺序依赖，严格顺序是次序造成的顺序
1. 输入函数参数比较杂乱，输入变量也是输出变量不够好


``` python
# action:   FOR,IF
# function:  Main,F1,F2,F3

def Main(x/x):
    FOR(x_1:0..1..x.len,F1(x,x_1))
def F1(x,x_1/x):
    FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
def F2(x,x_1,x_2/x):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x_If=x_If_1 and x_If_2
    IF(x_If,F3(x,x_1,x_2))
def F3(x,x_1,x_2/x):
    x_t=GV(x,x_1)
    SV(x,GV(x,x_2))
    SV(x,x_t)

```

#### 代码即需求，代码即结构

``` go
Ma[.] -> FOR(x_l,F1[...])
F1[.] -> FOR(x_2,F2[...])
F2[.] -> 1. 2. 3. 4. 5. IF(.,F3[...])
F3[.] -> 1. 2. 3.

```

#### 优化特征识别
``` go
FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
F2(..):
    ...//无x_2
    x_If_1=x_1<x_2
    ... && x_If

==>

FOR(x_2:x_1..1..x.len,F2(x,x_1,x_2))
F2(..):
    ...//无x_2
    //x_If_1=x_1<x_2
    ... && true

```

#### 灵感获得路径

#### 特征项获得是否满足条件，及改造方式的定义
```go
//运行及执行操作
//输入什么，代码片段，还是优化行
//特征提取方式近似于解析过程,正则？脚本？还是特殊执行体
def check1(F1[1:FOR..],x_2,F2[3:x_If_1..]):
  x_2 ==> 0..1..x.len
  if_pos //IF改变的位置
  x_2_toTrue //改变的代码位置
  while(getop(F1)):
    switch(op):
        FOR:
          //进入函数
          //没有其他x_2依赖项
        FUN:
          //没有其他x_2依赖项
        IF:
          if(x_1<x_2) //可作为条件，正好是F2的函数
          //
          //没有其他x_2依赖项
        Op:
          //其他执行体
          //没有其他x_2依赖项
          //使用x_1<x_2的位置记录下来
   return false or true
```

#### 以上结果偏复杂，难以逻辑组合、集成、改型
```python
def check1(F1[1:FOR..],x_2,F2[3:x_If_1..]):
    #产生遍历表
     '''产生x_2的判断逻辑的周期
     F1:
       FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
        -->x_1_val=GV(x,x_1),x_2_val=GV(x,x_2)
      | -->x_If_1=x_1<x_2 *
      | -->x_If_2=GV(x,x_2)<GV(x,x_1)
      | -->x_If=x_If_1 and x_If_2,IF(x_If,F3(x,x_1,x_2))
      ...产生依赖关系图
     
     '''
```
#### 产生结论
1. 是否有找到符合条件的项(有>0的某个x_2范围没有使用),并且所有执行导致输出都与x_2的范围有关，不存在其他导致输出的操作
2. FOR发生的位置，及IF发生的位置s
3. IF结果需要替换的位置s
4. 有一个不完备的地方，不出现x_2的部分可能引发输出，提取后无法保障输出的一致性，如果确保x_2改型后不导致引发问题？

#### 尝试优化为输入输出分离? 会变复杂化

``` python
# action:   FOR,IF
# function:  Main,F1,F2,F3

def Main(x/x):
    FOR(x_1:0..1..x.len,F1(x,x_1))
def F1(x,x_1/x):
    FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
def F2(x,x_1,x_2/x):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x=x
    x_If=x_If_1 
    x_If=x_If and x_If_2
    IF(x_If,F3(x,x_1,x_2))
def F3(x,x_1,x_2/x):
    x_t=GV(x,x_1)
    SV(x,GV(x,x_2))
    SV(x,x_t)

```



#### 变形表，包括生命周期和输入输出依赖
1. 定义不能对变量进行重复赋值操作，必须新建立变量来形成
2. 所有变量的赋值也是函数，也有输入，操作，输出
3. 需要定义生命周期问题，即此后访问当前变量值，而不是之前的
4. 变量属于函数内的部分，因此变量即是函数内的变量


``` python
# action:   FOR,IF
# function:  Main,F1,F2,F3


1. Data
# Main:x_1[B1]
# Main:x[B1]
# F1:x_1[B1]
# F1:x[B1]
# F1:x_2[B1]
# F2:x_1[B1]
# F2:x_2[B1]
# F2:x[B1]
# F2:x_If[B1]
# F2:x_If[B2]
# F2:x

2. Code[执行的实体,顺序性,是有变量重名保证的!!Good!]
#. IF导致不同的过程，如何处理变量名来源问题？ 
#. 消除else语句使得依序产生

GV:F2_x,F2_x_1->F2_x_1_val
GV:F2_x,F2_x_2->F2_x_2_val
...
F3:F2_x_1,F2_x_2,F2_x->F2_x

3. Run[依赖执行的完备性]
F2 <- F2_x
    <- F2:x[B1]
       =:x->x
    <- F2:x[B2]
       F3:x,x_1,x_2->x
         <- IF x_If->F3
            <- x_If[B2]
               <- x_If[B1]

def Main(x/x):
    FOR(x_1:0..1..x.len,F1(x,x_1))
def F1(x,x_1/x):
    FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
def F2(x,x_1,x_2/x):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x=x
    x_If=x_If_1
    x_If=x_If and x_If_2
    IF(x_If,F3(x,x_1,x_2))
def F3(x,x_1,x_2/x):
    x_t=GV(x,x_1)
    SV(x,GV(x,x_2))
    SV(x,x_t)

```



#### 方案1
<font size=1>

1. 思考属性
 - 重新仿真人脑的稀疏过程，而不是全递进过程
 - 当前数据流不会非常多，需要及时剔除和整合
 - 每个数据流是不完整的，包含了比较多的不定量(规则固定?扩充的?)
 - 需要产生当前分析的需求链
 - 同一个代码段可产生两个函数调用，这两个函数逻辑存在关联?
 - 降维到数据依赖空间，明显脱离了人的直观感受，人不是从输入来判断的


2. 原始代码
``` python
    #输入
    def FunA(x,y):
        FunB(x-1,y)
    def FunB(m,n):
        if(m>0,m+FunA(m,n))
        if(m==0,SV(n,0))
    def SV(k,l):
        k=0
    #目标
    def FunAF(x,y):
        y=x+(x-1)+...+1+0
         =x*(x+0)/2
```

3. 可产生的数据流碎片

``` python
#--- Dictionary 是否需求，还是仅仅是快表
x,y -> FunA -> FunA_R?
m,n -> FunB -> FunB_R?
FunB_m,FunB_n,m,n -> IFFunA -> Info #产生是非结构
Info I> FunA -> FunA_ret?  #选择执行（降维的If）
[Set] S> FunA -> FunA_ret?  #集合执行（降维的For）
k,l -> SV -> k
```

4. 构建临时堆栈结构

```python

#---Knowledge
x,y D> FunA -> FunA_R?
FunA_x-1,FunA_y ->FunB -> FunB_R?
FunB_m,FunB_n,m,n -> IF -> Info1 #产生是非结构
Info1,m,n I> FunA -> FunA_ret?  #选择执行（降维的If）
FunB_m,FunB_n,m,n -> IF -> Info2 #产生是非结构
Info2,n,0 I> SV -> SV_R?  #选择执行（降维的If）
k,l -> SV -> SV_R_k
#---中间过程
Info2,n,0 I> SV -> SV_R_k  #产生补充
m,n -> FunB -> FunB_R   #产生补充
x,y D> FunA -> FunA_R   #产生补充
#---临时归纳
x,y D -> FunA  #函数调用，即是变形调用
    -> FunA_x-1,FunA_y 
    -> FunB
    -> IF -> Info1
    -> m,n
    -> FunA 

#-----Problem
FunA_R?
FunB_R?
SV_R?

#---Action And History?
- Find FunA_R?
- Find FunB_R?
- Find SV_R?
- Slove SV_R
- Slove FunB_R
- Slove FunA_R

```

</font>

#### 方案2

- 原始代码
``` python
    #输入
    def FunA(x,y):
        FunB(x-1,y)
    def FunB(m,n):
        if(m>0,FunA(m,n))
        if(m==0,SV(m,n))
    def SV(k,l):
        l=k
    #目标
    def FunAF(x,y):
        y=0
```
- 分析过程

```python
#Problem
00. Optimize(FunA)

#Knowledge (正向结果）
02.[a,b] -> FunA(x,y) -> FunA_r[]
02. FunA_x,FunA_y -> FunB(m,n) -> FunB_r[]
04. [m,n] -> FunB(m,n) -> FunB_r[]
04. m>0 -> IF -> Info1
04. Info1,m,n I> FunA -> FunA_r[]
04. m>1 -> IF -> Info2
04. Info1,m,n I> SV -> SV_r[]
06. [k,l] -> SV -> SV_r[]
06. 0 -> assign -> k

#Guess1 (反向推理)
02. FunA_r[] <- FunB_r[]
04. FunB_r[] <- FunA_r[]
04. FunB_r[] <- SV_r[]
06. SV_r[] <- [k]
08. Ana_m <- [0,1,...]
10. m=0 -> k=0
12. m=1 -> FunA(1,n) -> FunB(0,n) -> SV(0,n) -> n=0
13. m -> k=0
14. FunA_x,FunA_y -> SV -> FunA_y

#Action
01. Look Def FunA
03. Look Def FunB
05. Look Def SV
07. Know Fun Process
09. Anayisis FunB m=0
11. Anayisis FunB m=1
13. Anayisis FunB Symbol
14. Define Def

#

```

1. 发现缺乏的是符号演算推理的智能过程，而不是简单的代码分析过程


In [16]:
#### Demo仿真测试
Raw_Code='''
def Main(x):
    x_1=0:1:x.len
    x=F1#FOR(x_1,x,x_1)
def F1(x,x_1):
    x_2=0:1:x.len
    x=F2#FOR(x_2,x,x_1,x_2)
def F2(x,x_1,x_2):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x= SV#IF(false,x,1,0)
    x= SV#IF(false,x,2,9)
    x_If=x_If_1 & x_If
    x_If=x_If_1
    x_If=x_If & x_If_2
    x=F3#IF(x_If,x,x_1,x_2)
def F3(x,x_1,x_2):
    x_t1=GV(x,x_1)
    x_t2=GV(x,x_2)
    x=SV(x,x_2,x_t1)
    x=SV(x,x_1,x_t2)
    x= F4#IF(false,x,x_1,x_2)
def F4(x,x_1,x_2):
    x= F5(x,x_1,x_2)
def F5(x,y_1,y_2):
    x= SV(x,y_1,y_2)
'''
#print((1,2,3)[1])

In [17]:
#### 尝试进行Data，Code分离
#### 消除else语句使得IF分为两条语句依序产生
#### 即是输入又是输出的必须显性表达，不够优秀，应该可以通过复杂算法实现自动判断
#### 递归存在问题?  存在比较严重的问题，导致无法形成有向图

import StringIO
import re

# 解析为行
buf = StringIO.StringIO(Raw_Code);Raw_Lines=buf.readlines();

# 当前函数名
Cur_Fun="";

Data=dict();
Code=list();

for Raw_l in Raw_Lines:
    Raw_l=Raw_l.strip();
    #print(Raw_l)
    Raw_p='';
    #判断是否为函数
    if(Raw_l.find("def")>=0):
        Cur_Fun=re.compile(r'def [^(]+\(').findall(Raw_l)[0][4:-1]
        Cur_Var_Param=re.compile(r'\([^)]+\)').findall(Raw_l)[0][1:-1].split(',')
        for Cur_Var_Param_i in Cur_Var_Param:
            Data[Cur_Fun+"@"+Cur_Var_Param_i]=0;
        Cur_Var_Param_r=list()
        for Cur_Var_Param_i in Cur_Var_Param:
            Cur_Var_Param_i=Cur_Var_Param_i.strip();
            Cur_Var_Param_Fun=Cur_Fun+"@"+Cur_Var_Param_i
            if(Cur_Var_Param_Fun in Data):
                Cur_Var_Param_r.append(Cur_Var_Param_Fun+"@"+str(Data[Cur_Var_Param_Fun])) 
        #print(Cur_Var_Param)
        Raw_p=("DF",Cur_Fun,Cur_Var_Param_r,list(""))
    else:
        if(Raw_l.find("=")>=0): #赋值操作
            D_n=Raw_l.find("=");
            Cur_Var_s=Raw_l[D_n+1:]
            Cur_Var_p=(Cur_Fun+"@"+Cur_Var_s)
            # 解析赋值来源的输入输出，并确定输入输出
            #print(Raw_l)
            Cur_Var_Param_r=list()
            #Cur_Var_Fun='Error'
            if(Cur_Var_s.find("(")>=0): #直接函数
                Cur_Var_Fun=re.compile(r'[^\(]+\(').findall(Cur_Var_s)[0][0:-1]
                #print(Cur_Var_Fun)
                Cur_Var_Param=re.compile(r'\([^)]+\)').findall(Cur_Var_s)[0][1:-1].split(',')
                
                for Cur_Var_Param_i in Cur_Var_Param:
                    Cur_Var_Param_i=Cur_Var_Param_i.strip();
                    Cur_Var_Param_Fun=Cur_Fun+"@"+Cur_Var_Param_i
                    if(Cur_Var_Param_Fun in Data):
                        Cur_Var_Param_r.append(Cur_Var_Param_Fun+"@"+str(Data[Cur_Var_Param_Fun])) #TODO:
                    else:
                        Cur_Var_Param_r.append(Cur_Var_Param_Fun); #TODO：函数也是需要有序号的
                #print(Cur_Var)
                Raw_p_M="=F";#+Cur_Var_Fun+"\t"+str(Cur_Var_Param_r)+"->"+str([Cur_Var+"@"+str(Data[Cur_Var])]))
            elif(Cur_Var_s.find("<")>=0 or Cur_Var_s.find("&")>=0 or True):
                #Cur_Var_Param_r=list()
                Cur_Var_Fun='SV'
                Cur_Var_Param=re.split('<|&',Cur_Var_s)
                #print("------->:",Cur_Var_Param)
                for Cur_Var_Param_i in Cur_Var_Param:
                    Cur_Var_Param_i=Cur_Var_Param_i.strip();
                    Cur_Var_Param_Fun=Cur_Fun+"@"+Cur_Var_Param_i
                    if(Cur_Var_Param_Fun in Data):
                        Cur_Var_Param_r.append(Cur_Var_Param_Fun+"@"+str(Data[Cur_Var_Param_Fun])) #TODO:
                    else:
                        Cur_Var_Param_r.append(Cur_Var_Param_Fun); #TODO：函数也是需要有序号的
                Raw_p_M="=<";#+Cur_Var_Fun+"\t"+str(Cur_Var_Param_r)+"->"+str([Cur_Var+"@"+str(Data[Cur_Var])]))
        
            #记录赋值的变量
            Cur_Var=(Cur_Fun+"@"+Raw_l[0:D_n])
            if(Cur_Var in Data):
                Data[Cur_Var]+=1; #进行生存期累计
            else:
                Data[Cur_Var]=0;#="" #TODO，判断是否第2次
            Raw_p=(Raw_p_M,Cur_Var_Fun.strip(),Cur_Var_Param_r,[Cur_Var+"@"+str(Data[Cur_Var])])

        elif(Raw_l.find("(")>=0):                   #其他都是函数操作
            Cur_Var_Fun=re.compile(r'[^\(]+\(').findall(Raw_l)[0][0:-1]
            Cur_Var_Param=re.compile(r'\([^)]+\)').findall(Raw_l)[0][1:-1].split(',')
            Cur_Var_Param_r=list()
            for Cur_Var_Param_i in Cur_Var_Param:
                    Cur_Var_Param_i=Cur_Var_Param_i.strip();
                    Cur_Var_Param_Fun=Cur_Fun+"@"+Cur_Var_Param_i
                    if(Cur_Var_Param_Fun in Data):
                        Cur_Var_Param_r.append(Cur_Var_Param_Fun+"@"+str(Data[Cur_Var_Param_Fun])) 
            Raw_p=("|F",Cur_Var_Fun.strip(),Cur_Var_Param_r,list(""))
            pass
    if(len(Raw_p)>=4):
        Code.append(Raw_p)
    #print(Raw_l+"\r\n".ljust(16,' ')+str(Raw_p))

In [18]:
#### 尝试弥补函数的输出表
#def AddCodeOut()

import copy
Code_All=copy.deepcopy(Code)
for Code_i in Code_All:
    if(Code_i[0]=="DF"):
        Code_in=Code_i[2]
        Code_ou=Code_i[3]
        for Code_in_i in Code_in:
            Code_in_i_val=re.compile(r'^[^@]+@[^@]+').findall(Code_in_i)[0]
            Code_in_i_val_new=Code_in_i_val+"@"+str(Data[Code_in_i_val]);
            
            if(Code_in_i!=Code_in_i_val_new):
                #pass
                #print(Code_in_i,Code_in_i_val_new)
                Code_i[3].append(Code_in_i_val_new)

for Code_i in Code_All:
    Code_s=Code_i[0].ljust(16,' ');
    Code_s+=Code_i[1].ljust(16,' ');
    Code_2_r="";
    for Code_2 in Code_i[2]:
        Code_2_r+=Code_2+','
    Code_3_r="";
    for Code_3 in Code_i[3]:
        Code_3_r+=Code_3+','
    Code_s+=Code_2_r.ljust(50,' ');
    Code_s+=Code_3_r.ljust(50,' ');
    print(Code_s)

DF              Main            Main@x@0,                                         Main@x@1,                                         
=<              SV              Main@0:1:x.len,                                   Main@x_1@0,                                       
=F              F1#FOR          Main@x_1@0,Main@x@0,Main@x_1@0,                   Main@x@1,                                         
DF              F1              F1@x@0,F1@x_1@0,                                  F1@x@1,                                           
=<              SV              F1@0:1:x.len,                                     F1@x_2@0,                                         
=F              F2#FOR          F1@x_2@0,F1@x@0,F1@x_1@0,F1@x_2@0,                F1@x@1,                                           
DF              F2              F2@x@0,F2@x_1@0,F2@x_2@0,                         F2@x@3,                                           
=F              GV              F2@x@0,F2@x_1@0,                     

In [19]:
#进行反向依赖性遍历
                
Find_Need=list(['F4@x@1'])
Find_Outp=set(['F4@x@0','F4@x_1@0','F4@x_2@0'])

NodeStr=''
NodeLink = '';

print('Need:',Find_Need,Find_Outp);print('');

while len(Find_Need)>0:
    Find_Need_cur=Find_Need.pop()
    
    #如果当前输出还不在Find_Outp里面,寻找相关项放入
    print("Getd:",Find_Need_cur)
    if Find_Need_cur not in Find_Outp:
        isFind=0;
        for Code_i in Code_All:
            if(Code_i[0].find("DF")>=0):
                continue;
            if Find_Need_cur in Code_i[3]:
                isFind=1
                #print("Sear:",Code_i)
                FunName='';
                FunFrom='';FunTo=Find_Need_cur;
                if(Code_i[0].find("=<")>=0):
                    for Code_2_Num in range (0,len(Code_i[2])):
                        FunFrom+=Code_i[2][Code_2_Num]+','
                        NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%(Code_i[2][Code_2_Num],'P|'+FunTo)
                        Find_Need.append(Code_i[2][Code_2_Num])
                        
                    #print('Need:',Find_Need)
                if(Code_i[0].find("=F")>=0):
                    if(Code_i[1].find("#")>=0):#当前是FOR或IF函数调用，则加入输入参数需求
                        FunName=Code_i[1][0:Code_i[1].find("#")] ;#函数名
                        for Code_2_Num in range (1,len(Code_i[2])):
                            FunFrom+=Code_i[2][Code_2_Num]+','
                            NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%(Code_i[2][Code_2_Num],'P|'+FunTo)
                            if(Code_i[2][Code_2_Num] not in Find_Outp): #不在已经获得的输出中
                                Find_Need.append(Code_i[2][Code_2_Num])
                                
                    else:                     #普通函数调用
                        FunName=Code_i[1];
                        for Code_2_Num in range (0,len(Code_i[2])):
                            FunFrom+=Code_i[2][Code_2_Num]+','
                            NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%(Code_i[2][Code_2_Num],'P|'+FunTo)
                            if(Code_i[2][Code_2_Num] not in Find_Outp): #不在已经获得的输出中
                                Find_Need.append(Code_i[2][Code_2_Num])
                        #print('''{"from":"%s", "to":"%s", "text":"Out", "curviness":20},'''%('P|'+FunTo,FunTo))
                
                NodeStr+='''{"id":"%s",  "text":"%s"},'''%('P|'+FunTo,'P|'+FunTo)
                #NodeLink+='''{"from":"%s", "to":"%s", "text":"Out", "curviness":20},'''%('P|'+FunTo,FunTo)
                
                print(FunFrom,'->',FunName,'->',FunTo)
                
                if(len(FunName)>0):#当前是函数调用，得加入函数定义需求
                    for Code_ii in Code_All:
                        if(Code_ii[0].find("DF")>=0 and Code_ii[1]==FunName):#找到函数
                            #print("FiFR:",Code_ii)
                            for Code_ii_Num in Code_ii[2]:
                                NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%('P|'+FunTo,Code_ii_Num)
                                Find_Outp.add(Code_ii_Num)
                            for Code_ii_Num in Code_ii[3]:
                                Find_Need.append(Code_ii_Num)
                                NodeLink+='''{"from":"%s", "to":"%s", "text":"Out", "curviness":20},'''%(Code_ii_Num,FunTo)
                            if(len(Code_ii[3])>0):
                                NodeLink+='''{"from":"%s", "to":"%s", "text":"Out", "curviness":20},'''%('P|'+Code_ii[3][0],Code_ii[3][0])
                Find_Outp.add(Find_Need_cur)
                print('Need:',Find_Need,Find_Outp);print('');
        if(isFind==0):
            pass;#print("Erro:",Find_Need_cur)
    else:
        print("Erro:",Find_Need_cur)
        for Code_i in Code_All:
            if(Code_i[0].find("DF")>=0 and  Find_Need_cur in Code_i[3]):
                #print(Code_i)
                for Code_2_Num in range (0,len(Code_i[2])):
                    print('''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%(Code_i[2][Code_2_Num],'R|'+Find_Need_cur))
                    NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%(Code_i[2][Code_2_Num],'R|'+Find_Need_cur)
                NodeStr+='''{"id":"%s",  "text":"%s"},'''%('R|'+Find_Need_cur,'R|'+Find_Need_cur)
                NodeLink+='''{"from":"%s", "to":"%s", "text":"In", "curviness":20},'''%('R|'+Find_Need_cur,Find_Need_cur)

('Need:', ['F4@x@1'], set(['F4@x@0', 'F4@x_2@0', 'F4@x_1@0']))

('Getd:', 'F4@x@1')
('F4@x@0,F4@x_1@0,F4@x_2@0,', '->', 'F5', '->', 'F4@x@1')
('Need:', ['F5@x@1'], set(['F5@y_2@0', 'F4@x_1@0', 'F4@x@0', 'F5@x@0', 'F4@x_2@0', 'F4@x@1', 'F5@y_1@0']))

('Getd:', 'F5@x@1')
('F5@x@0,F5@y_1@0,F5@y_2@0,', '->', 'SV', '->', 'F5@x@1')
('Need:', [], set(['F5@y_2@0', 'F4@x_1@0', 'F4@x@0', 'F5@x@0', 'F4@x_2@0', 'F5@x@1', 'F4@x@1', 'F5@y_1@0']))



In [20]:
# 改善节点和链路
for node in Find_Outp:
    NodeStr+='''{"id":"%s",  "text":"%s"},'''%(node,node)
NodeStr=NodeStr[0:-1]
NodeLink=NodeLink[0:-1]

#print("NodeStr:",NodeStr)
#print("NodeLink:",NodeLink)

In [21]:
## 示例数据
'''
{ "class": "GraphLinksModel",
  "nodeKeyProperty": "id",
  "nodeDataArray": [ 
{"id":"P|F3@x@3", "text":"P|F3@x@3", "loc":"-133 0"},
{"id":"P|F3@x@2", "text":"P|F3@x@2", "loc":"10.97542319635673 71"},
{"id":"P|F3@x_t2@0", "text":"P|F3@x_t2@0", "loc":"108.95084639271346 189"},
{"id":"P|F3@x@1", "text":"P|F3@x@1", "loc":"270.11451726973434 143"},
{"id":"P|F3@x_t1@0", "text":"P|F3@x_t1@0", "loc":"459 228"},
{"id":"F3@x_1@0", "text":"F3@x_1@0", "loc":"-235.58632912297918 325.2"},
{"id":"F3@x_t2@0", "text":"F3@x_t2@0", "loc":"20.56572981556502 145"},
{"id":"F3@x@3", "text":"F3@x@3", "loc":"-237.1503295320236 -11.999999999999986"},
{"id":"F3@x@2", "text":"F3@x@2", "loc":"-58 43.00000000000003"},
{"id":"F3@x@1", "text":"F3@x@1", "loc":"138.09569297174738 109.00000000000003"},
{"id":"F3@x@0", "text":"F3@x@0", "loc":"216.09138594349474 325.25"},
{"id":"F3@x_t1@0", "text":"F3@x_t1@0", "loc":"368.2870789152419 175.00000000000003"},
{"id":"F3@x_2@0", "text":"F3@x_2@0", "loc":"62.85000000000002 327.20000000000016"}
 ],
  "linkDataArray": [ 
{"from":"F3@x@2", "to":"P|F3@x@3", "text":"In", "curviness":20, "points":[-57.999514520765615,64.35972843911163,-75.57726612259125,64.81192392561191,-84.01100148422404,56.28909311568613,-83.50136999529526,38.96683852311962]},
{"from":"F3@x_1@0", "to":"P|F3@x@3", "text":"In", "curviness":20, "points":[-181.19127913129776,325.20875928025623,-168.14491284035705,260.8851663135531,-143.56737071300094,176.62937117581342,-91.0141734421173,38.93334126463305]},
{"from":"F3@x_2@0", "to":"P|F3@x@3", "text":"In", "curviness":20, "points":[98.02887497769433,327.3097941722246,27.562070221780836,238.89643014990898,-30.14170390475065,142.78921286599268,-75.10241991792999,38.92574668984909]},
{"from":"P|F3@x@3", "to":"F3@x@3", "text":"Out", "curviness":20, "points":[-102.47301824116609,0.14295891229984314,-119.13348642399272,-16.66208560219507,-138.04642967573156,-20.29697087978485,-159.74913070956524,-10.065713573521428]},
{"from":"F3@x@1", "to":"P|F3@x@2", "text":"In", "curviness":20, "points":[138.09995773491187,134.0758892328661,111.90421094384133,137.65557098075956,91.11298956461282,129.33380584307884,75.47025094113431,109.86215877006944]},
{"from":"F3@x_1@0", "to":"P|F3@x@2", "text":"In", "curviness":20, "points":[-162.74670583814563,325.3713184216839,-125.97164987067988,293.6992744444203,-54.52656719640432,221.73602810167003,43.46016678531271,109.84251311232188]},
{"from":"F3@x_t2@0", "to":"P|F3@x@2", "text":"In", "curviness":20, "points":[53.48737160912151,145.15450320519713,46.435044899475926,138.04398980466715,43.917696947914386,127.70485232900194,51.60954289950373,109.92186354795055]},
{"from":"P|F3@x@2", "to":"F3@x@2", "text":"Out", "curviness":20, "points":[11.963993169598469,100.53144640710408,-5.3898903590312965,103.77294223243909,-13.531661021905698,96.38448737691246,-14.101091496532582,81.96564061873899]},
{"from":"F3@x@0", "to":"P|F3@x_t2@0", "text":"In", "curviness":20, "points":[235.4618603462507,325.43001614442926,202.53077365475903,297.5511129699979,182.25887071662348,264.61740038593183,173.49252804122133,227.9542168112424]},
{"from":"F3@x_2@0", "to":"P|F3@x_t2@0", "text":"In", "curviness":20, "points":[113.23478141824248,327.20006585405156,112.42267350288996,286.5867516306894,125.6293869485981,253.4040199115908,150.5639480176039,227.82848849201207]},
{"from":"P|F3@x_t2@0", "to":"F3@x_t2@0", "text":"Out", "curviness":20, "points":[108.95150603135698,205.39418028563304,100.07228027757462,205.22955866968906,90.47523788531515,199.0674079188853,83.01644747736196,183.91777870953135]},
{"from":"F3@x@0", "to":"P|F3@x@1", "text":"In", "curviness":20, "points":[260.2728599802943,325.2517486815433,266.135452275934,265.6658253671779,283.34607141889904,217.05249821427878,306.4976166459968,181.88115254763136]},
{"from":"F3@x_2@0", "to":"P|F3@x@1", "text":"In", "curviness":20, "points":[127.87998606377833,327.2979173036219,173.66088128852397,264.65634079675516,226.84955715322934,216.7320048930689,286.9484244262996,181.72063377429376]},
{"from":"F3@x_t1@0", "to":"P|F3@x@1", "text":"In", "curviness":20, "points":[369.0625944789381,203.73906510262992,347.96993537299227,207.2065631728573,334.26919944699426,199.2467732031912,327.27276022395324,181.9319695117415]},
{"from":"P|F3@x@1", "to":"F3@x@1", "text":"Out", "curviness":20, "points":[270.601655537096,170.6705188966392,240.20316323982513,175.6330093403522,217.24567989103846,167.72497087840767,198.95310240755404,147.82999078423262]},
{"from":"F3@x@0", "to":"P|F3@x_t1@0", "text":"In", "curviness":20, "points":[288.92403413266607,325.61003069113826,334.2627786393041,297.5986572200742,383.20379717810437,278.25852115217725,461.7453650464258,260.3870855602916]},
{"from":"F3@x_1@0", "to":"P|F3@x_t1@0", "text":"In", "curviness":20, "points":[-134.57617192768367,336.85085929884485,94.1386859642983,301.16175363675967,220.39724238608218,283.80146694678086,459.2495749363875,254.78146737922546]},
{"from":"P|F3@x_t1@0", "to":"F3@x_t1@0", "text":"Out", "curviness":20, "points":[459.0006552639768,244.4044611207852,440.7195316798105,242.8385540612709,429.4478064145311,233.28309593277393,425.46708393253493,213.95642318290365]}
 ]}
''';

In [23]:
# 定义和展现数据

## 目前测试不是非常适合用有向图解决，因为缺乏函数或变量的生存时间性质等

showJson1='''
{ "class": "GraphLinksModel",
  "nodeKeyProperty": "id",
  "nodeDataArray": [ 
%s
 ],
  "linkDataArray": [ 
%s
 ]}
'''%(NodeStr,NodeLink)

# 展示网站
from IPython.display import HTML
f = open('Show.html','rb')
data = f.read()                   # 读取文件内容
f.close()
#HTML(data.replace("showJson1",showJson1))

In [337]:
print('\033[31ms')
print('\033[0m')

s



# 三、初步实现与建模

## 1 模型转换
#### 定义过程型
|P4Short|P     |复杂度  |  描述|
|----|----     |---  |----|
|FR  |ForRange |x   |   遍历输入量x产生输出集合0..1..x,作为一个块看待,可输出ind，以及标识结束，作为后续依赖的标志|
|IS  |IfSmall  |1   |   判断是否左x1小于右侧x2|
|GS  |GetSize  |1   | 获得x集合的长度|
|GV  |GetValue |1   | 获得x结合的indx位置的值|
|SV  |SetValue |1   | 设置x结合的indx位置的值|
|LA  |LogicAnd |1   | 逻辑与运算|
|SW  |swap     |1  | 交换x的ind1和ind2位置
|IM  |InsertMap|log(x)    |插入Map数据结构|
|GM  |GetMap|1    |获得Map结构中的一项|

|说明|
|:---|
| 因此依赖关系针对FR分为两种，一种为获得其中一项作为FR的内部依赖，一种为FR的结束依赖|
| FR是函数的抽象，普通函数为FR为1的函数|
|函数本身也决定了一种独特的依赖关系，内部依赖和外部依赖，是否可以将两种依赖统一呢|
|目前两种依赖关系等效于：即非树形，也非完全的AST语法树|

#### 过程输入输出表（图?CNN?）

##### P2，P3输入输出上没有体现出依赖关系，但是实质是存在依赖性的，即P3必须依赖于P2，是否依赖关系要重新建立一张表呢？

|N  |P    |P_pre  |    x|  x_len|   x_1|   x_2| x_1_val |x_2_val | x_ind_If | x_val_If| x_If|
|---|---  |---    |---  |---   |---   |---   |---    |---     |---  | ----|  ----|
|  1| GS  |0     |   I  |    O |
|  2| FR  |1     |&nbsp; |    I |    O|
|  3| FR  |2     |&nbsp; |    I |&nbsp;|    O|
|  4| GV  |3     | I    |&nbsp; | I   |&nbsp;| O|   
|  5| GV  |3     | I    |&nbsp; |&nbsp;|I    | &nbsp; |       O|
|  6| IS  |3     | &nbsp;| &nbsp;|   I |    I| &nbsp; |  &nbsp;  |    O|
|  7| IS  |3     | &nbsp;| &nbsp;|&nbsp;|&nbsp;| I     |  I |   &nbsp;|  O |
|  8| LA  |4,5,6,7 | &nbsp;| &nbsp;|&nbsp;|&nbsp;| &nbsp;  |&nbsp; |   I|  I | O|
|  9| SW  |8     | O    |&nbsp; |I    | I   | &nbsp; | &nbsp;   |&nbsp;|  &nbsp;|I|

#### 依赖关系表
|N  |P1    |P2    |P3    |P4    |P5    |P6    |P7    |P8
|---|---  |---    |---  |---   |---   |---   |---    |--- |
| P1|
| P2|Y|
| P3|&nbsp;|Y|
| P4|&nbsp;|Y|
| P5|&nbsp;|&nbsp;|Y|
| P6|&nbsp;|Y|Y|
| P7|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P8|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P9|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|


## 2 复杂度计算

``` go
P1 = 1
P2 = P1 * (x_len) 
P3 = P2 * (x_len)
P4 = P3 * (1)
P5 = P3 * (1)
P6 = P2 + P3
P7 = P4 + P5
P8 = P6 + P7
P9 = P8

==>

P9 = P2 + P3 + P4 + P5
  = P1 * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len)
  = P1 * (x_len) * (x_len)
  ~= N * (x_len) * (x_len)
```